# SIADS 516 Homework 4: Spark SQL
Version 1.0.20200221.1
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

This homework assignment uses the Yelp Academic dataset, with which you should now be familiar.
We have created a few cells to get you started, but you're largely on your own to devise solutions to the
"real-world" questions below.

The best solutions will use spark.sql() calls as a preferred way to query the dataset and also use the fewest number of steps.  For example, to find the answer to "How many users have more than 100 "cool" votes?", this:
```
query = """
SELECT count(*) FROM user WHERE cool > 100
"""
spark.sql(query).show()
```
is preferable to:
```
user.filter('cool > 100').show()
```
or 
```
query = """
SELECT * FROM user
"""
df = spark.sql(query)
df.filter('cool > 100').show()
```
(Note that the last number is somewhat ridiculous.)

Our usual Spark mantra:

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('My First Spark application') \
    .getOrCreate() 

sc = spark.sparkContext

Load the JSON files:

In [2]:
business = spark.read.json('data/yelp_academic/yelp_academic_dataset_business.json.gz')
checkin = spark.read.json('data/yelp_academic/yelp_academic_dataset_checkin.json.gz')
review = spark.read.json('data/yelp_academic/yelp_academic_dataset_review.json.gz')
tip = spark.read.json('data/yelp_academic/yelp_academic_dataset_tip.json.gz')
user = spark.read.json('data/yelp_academic/yelp_academic_dataset_user.json.gz')

Create temp views for the DataFrames:

In [3]:
business.createOrReplaceTempView("business")
checkin.createOrReplaceTempView("checkin")
tip.createOrReplaceTempView("tip")
review.createOrReplaceTempView("review")
user.createOrReplaceTempView("user")

## Q1. How many users have more than 500 fans?

In [4]:
#user.printSchema()
query1 = 'SELECT count(*) AS user_count FROM USER WHERE fans > 500'
spark.sql(query1).show()

+----------+
|user_count|
+----------+
|       185|
+----------+



185 users have more than 500 fans

## Q2. How many businesses from Madison, Wisconsin are represented in the dataset?

In [5]:
query2 = """
SELECT count(*) FROM business
WHERE upper(city) = 'MADISON'
  AND upper(state) = 'WI'
"""
spark.sql(query2).show()

+--------+
|count(1)|
+--------+
|    3494|
+--------+



There are 3,494 businesses from Madison, Wisconsin represented in the dataset

## Q3: How many users have more than 500 fans?

In [6]:
query3 = 'SELECT count(*) AS user_count FROM USER WHERE fans > 500'
spark.sql(query3).show()

+----------+
|user_count|
+----------+
|       185|
+----------+



185 users have more than 500 fans

## Q4: Which US states are represented in the data set?  Use full names of states (you will need to look up the list of state abbreviations is you don't know them).

In [7]:
state_tuples = [("AL","Alabama"),("AK","Alaska"),("AZ","Arizona"),("AR","Arkansas"),("CA", "California"),("CO", "Colorado"),
                 ("CT","Connecticut"),("DC","Washington DC"),("DE","Delaware"),("FL","Florida"),("GA","Georgia"),
                 ("HI","Hawaii"),("ID","Idaho"),("IL","Illinois"),("IN","Indiana"),("IA","Iowa"),("KS","Kansas"),
                 ("KY","Kentucky"), ("LA","Louisiana"),("ME","Maine"),("MD","Maryland"),("MA","Massachusetts"),
                 ("MI","Michigan"),("MN","Minnesota"),("MS","Mississippi"),("MO","Missouri"),("MT","Montana"),
                 ("NE","Nebraska"),("NV","Nevada"),("NH","New Hampshire"), ("NJ","New Jersey"),("NM","New Mexico"),
                 ("NY","New York"),("NC","North Carolina"),("ND","North Dakota"),("OH","Ohio"), ("OK","Oklahoma"),
                 ("OR","Oregon"),("PA","Pennsylvania"),("RI","Rhode Island"),("SC","South Carolina"),("SD","South Dakota"),
                 ("TN","Tennessee"),("TX","Texas"),("UT","Utah"),("VT","Vermont"),("VA","Virginia"),("WA","Washington"),
                 ("WV","West Virginia"), ("WI","Wisconsin"),("WY","Wyoming")]

states_df = spark.createDataFrame(state_tuples, ['abbrev', 'name'])
states_df.createOrReplaceTempView("states_df")

query4 = """
SELECT distinct s.name
FROM business b
INNER JOIN states_df s ON s.abbrev = b.state
ORDER BY s.name
"""
spark.sql(query4).show(50)

+--------------+
|          name|
+--------------+
|       Alabama|
|        Alaska|
|       Arizona|
|      Arkansas|
|    California|
|   Connecticut|
|       Florida|
|       Georgia|
|      Illinois|
|      Nebraska|
|        Nevada|
|    New Jersey|
|    New Mexico|
|      New York|
|North Carolina|
|          Ohio|
|  Pennsylvania|
|South Carolina|
|     Tennessee|
|         Texas|
|          Utah|
|       Vermont|
|      Virginia|
|    Washington|
|     Wisconsin|
+--------------+



The above states are included in the business dataset

## Q5: What is the text of the funniest review?

In [8]:
query5 = 'SELECT text as funniest_review FROM review ORDER BY funny desc'
spark.sql(query5).first()

Row(funniest_review="Flew to Arizona a few months ago to try this. Disappointed that this place closed after Gordon Ramsay's return. Their food was one of the best food I had in my whole entire life!! Their caesar salad was great, sauce is very good. Their pizza was THE BEST I HAD IN MY WHOLE ENTIRE LIFE. From the garlic crust to the extremely cheesy topping, the pizza was absolutely outstanding. Wish I could come back but its now closed.")

The text of the funniest review is above

## Q6: Which review(s) has/have the most words?  What do you notice about the maximum word count?

In [9]:
#First pulled the max character length to see that this maximum is 5,000
query6 = """
SELECT review_id, review_length from
   (SELECT review_id, length(text) as review_length FROM review) r
    INNER JOIN (SELECT MAX(length(text)) as max_length FROM review) m on m.max_length = r.review_length
"""
#spark.sql(query6).show(10)




#To then get a maximum word count
from pyspark.sql.functions import udf, size, split
from pyspark.sql.types import IntegerType

def word_count(x):
    return len(x.split())

wordcount_udf = udf(lambda x: word_count(x), IntegerType())
spark.udf.register('wordcount_udf', word_count, IntegerType())

query6_2 = """
SELECT review_id, word_length as longest_review FROM 
((SELECT review_id, text, wordcount_udf(text) as word_length FROM review) r
  INNER JOIN (select MAX(wordcount_udf(text)) as max_words from review) m on m.max_words = r.word_length)
ORDER BY word_length desc
"""
spark.sql(query6_2).show(10)

+--------------------+--------------+
|           review_id|longest_review|
+--------------------+--------------+
|6-7Krt1orAWzvEEGw...|          1056|
+--------------------+--------------+



The query above gives the review ID that is considered the longest within the Yelp dataset. From this query (and the commented out query, as well), it was found that the longest review word count wise was 1,056 words and that Yelp has a 5,000 character limit.

## Q7: What are the names of the top 10 users who provided the most tips?

In [10]:
query7 = """
SELECT name from
(SELECT u.name
      , u.user_id
      , count(*) as tips 
FROM tip t
INNER JOIN user u on u.user_id = t.user_id
GROUP BY u.name, u.user_id
ORDER BY count(*) desc)
"""

spark.sql(query7).show(10)

+--------+
|    name|
+--------+
|    Momo|
|Jennifer|
|Samantha|
|  Daniel|
|Christie|
|     May|
|   Kurdy|
| Anthony|
| Shirley|
| Cherrie|
+--------+
only showing top 10 rows



The names of the top 10 users who provided the most tips are above

## Q8: List the names, number of reviews of businesses in Arizona and total number of reviews of the top 5 users (as determined by who has created the most number of reviews of businesses in Arizona).  Include a column that shows the percentage of reviews that are of businesses from Arizona.  The first row of the results should be:
```
+--------+--------+-----------+---------+
|    name|az_count|total_count|  percent|
+--------+--------+-----------+---------+
|    Brad|    1637|       1642|99.695496|
+--------+--------+-----------+---------+
```

In [11]:
query8 = """
SELECT name, az_count, total_count, percent from
(SELECT u.name
     , u.user_id
     , sum(case when b.state = 'AZ' then 1 else 0 end) as az_count
     , u.review_count as total_count
     , sum(case when b.state = 'AZ' then 1 else 0 end)/u.review_count as percent
FROM business b
INNER JOIN review r on r.business_id = b.business_id
INNER JOIN user u on u.user_id = r.user_id
GROUP BY u.name, u.user_id, u.review_count)
ORDER BY az_count desc
"""
spark.sql(query8).show(5)

+--------+--------+-----------+-------------------+
|    name|az_count|total_count|            percent|
+--------+--------+-----------+-------------------+
|    Brad|    1637|       1642| 0.9969549330085262|
|   Karen|    1559|       2340| 0.6662393162393162|
|Jennifer|    1250|       1929| 0.6480041472265422|
|    Gabi|    1151|       1932| 0.5957556935817805|
|Jennifer|    1059|       4190|0.25274463007159903|
+--------+--------+-----------+-------------------+
only showing top 5 rows



The above results display the names and number of reviews of the top 5 users who reviewed the most businesses in Arizona. 